# Data Science Notebook 

In [2]:
import os, sys
import warnings
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [3]:
# Turn off jedi
%config IPCompleter.use_jedi=False

workdir = os.path.dirname(os.getcwd())

sys.path.append(workdir)

pd.options.display.max_columns = None

warnings.filterwarnings("ignore")

display(
    HTML(
        """
        <style>
        .container { width:100% !important; }
        </style>
        """
    )
)

In [ ]:
from pc_zap_scrapper.scrap import search_estates
from pc_zap_scrapper import ACTION, LOCALIZATION, TYPE

search_estates(ACTION, TYPE, LOCALIZATION)